In [6]:
#instalacja pulp

!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

Reading package lists... Done
Building dependency tree       
Reading state information... Done
coinor-cbc is already the newest version (2.9.9+repack1-1).
coinor-clp is already the newest version (1.16.11+repack1-1).
glpk-utils is already the newest version (4.65-1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


Biblioteka Pulp pozwala również na analzię obwodów elektrycznych i wyznaczanie maksymalnych wartości prąd dla poszczególnych gałęzi układu. Układ podany na zajęciach zawiera rezystancje o danych wartosciach R1=8Ω	R2=6Ω	R3=4Ω	R4=10Ω	R5=8Ω oraz określono limit prądu mogącego przepłynąć przez rezystory R1-2A	R2-3A	R3-4A	R4-2A	R5-2A

In [8]:
from pulp import *
PradUkladu = LpProblem("Prad_ukladu_rezystorow", LpMaximize)
NazwyGalezi = ["R1", "R2", "R3", "R4", "R5"]
Rezystancje = { "R1": 8, "R2" : 6, "R3" : 4, "R4" : 10, "R5" : 8 }
LimitPradow = { "R1": 2, "R2" : 3, "R3" : 4, "R4" : 2,  "R5" : 2 }

PlynacePrady = LpVariable.dicts("Prad", NazwyGalezi, 0)
PradUkladu += PlynacePrady["R3"], "Prad_calego_ukladu"
# limity prądów poszczególnych gałęzi
for iRx in PlynacePrady:
    PradUkladu += PlynacePrady[iRx] <= LimitPradow[iRx]
# prądowe prawo Kirchhoffa
PradUkladu += PlynacePrady["R1"] + PlynacePrady["R2"] == PlynacePrady["R3"]
PradUkladu += PlynacePrady["R4"] + PlynacePrady["R5"] == PlynacePrady["R3"]
# ograniczenie prądów w gałęziach równoległych wynikające z dzielnika rezystancyjnego
PradUkladu += PlynacePrady["R2"] == PlynacePrady["R1"] * Rezystancje["R1"] / Rezystancje["R2"]
PradUkladu += PlynacePrady["R4"] == PlynacePrady["R5"] * Rezystancje["R5"] / Rezystancje["R4"]
# zbędne (redundantne) ograniczenia w gałęziach przeciwległych
#PradUkladu += PlynacePrady["R1"] == PlynacePrady["R2"] * Rezystancje["R2"] / Rezystancje["R1"]
#PradUkladu += PlynacePrady["R5"] == PlynacePrady["R4"] * Rezystancje["R4"] / Rezystancje["R5"]
# rozwiązanie
PradUkladu.solve()
print("Status rozwiazania: ", LpStatus[PradUkladu.status])
print("Układ można obciążyć prądem max. {:.2f} A".format(PradUkladu.objective.value()))
for var in PradUkladu.variables():
    print("{}: {:.2f} A".format(var.name, var.value()))

Status rozwiazania:  Optimal
Układ można obciążyć prądem max. 3.60 A
Prad_R1: 1.54 A
Prad_R2: 2.06 A
Prad_R3: 3.60 A
Prad_R4: 1.60 A
Prad_R5: 2.00 A


Kolejnym zadaniem było wykorzystanie biblioteki pulp w celu dobrania wartości rezystancji mostka tak aby straty mocy były najmniejsze. W danej gałęzu przy znanym napięciu U ma popłynąć zadany prąd z z określoną dokładnością.Wartości napięć: U1=6V	U2=10V	U3=4V	U4=7V	U5=3V Wartości prądów I1=4A I2=2A I3=2A I4=2A I5=4A


In [11]:
prob = LpProblem("Minimalizacja mocy utraconej",LpMinimize)

I1=LpVariable("I1",3,5)
I2=LpVariable("I2",1,3)
I3=LpVariable("I3",1,3)
I4=LpVariable("I4",1,3)
I5=LpVariable("I5",3,5)

U = (6.0, 10.0, 4.0, 7.0, 3.0) # wartość napięć na poszczególnych rezystorach Ui z tabeli podanej na zajęciach

prob += (I1 * U[0]) + (I2 * U[1]) + (I3 * U[2]) + (I4 * U[3]) + (I5 * U[4]), "Moc całkowita"

#ograniczenia wynikające z praw Kirchhoff'a
prob += I1 - I3 == I4, "oczko górne"
prob += I2 + I3 == I5, "oczko dolne"

prob.writeLP("Moce.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue, "mA")
print()
print("Straty:",value(prob.objective),"mW")  
print()


for v, x in zip(U,prob.variables()):
  print("R", U.index(v)+1, "=", (v/x.varValue), "kOhm")
#wydruk wartości poszczególnych rezystorów


Status: Optimal
I1 = 3.0 mA
I2 = 1.0 mA
I3 = 2.0 mA
I4 = 1.0 mA
I5 = 3.0 mA

Straty: 52.0 mW

R 1 = 2.0 kOhm
R 2 = 10.0 kOhm
R 3 = 2.0 kOhm
R 4 = 7.0 kOhm
R 5 = 1.0 kOhm


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
